In [1]:
import numpy as np
import pandas as pd
from itertools import islice
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

/Users/franzmornau/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/franzmornau/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
%run ../src/acc_df.py

In [3]:
Ac = Acc_data('../data/cleaned/coll_coord.csv')

In [4]:
acc = Ac.get_all_coll() #['']

#### Time series by week

In [10]:
acc_04_18 = acc[(acc['year'] != 2019) & (acc['year'] != 2003 )]
acc_w = acc_04_18[['year','week','OBJECTID']].groupby(['year','week']).count()
week_numbers = acc_w.reset_index()
week_numbers.rename({'OBJECTID': 'acc_nr'}, axis=1, inplace=True)
week_numbers.head(2)

,year,week,acc_nr
0,2004,1,142
1,2004,2,263


In [11]:
week_numbers.acc_nr.sum()/len(week_numbers)

252.49236641221373

### other data / dummies

In [12]:
week_d = pd.get_dummies(week_numbers.week).join(week_numbers)
week_d['year_red']= week_d.year.apply(lambda x: x-2004)
week_d.drop(['week','year'], axis=1, inplace=True)
week_d['week_avg']=252.49
week_d.tail()

,1,2,3,4,5,6,7,8,9,10,...,47,48,49,50,51,52,53,acc_nr,year_red,week_avg
781,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,261,14,252.49
782,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,274,14,252.49
783,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,275,14,252.49
784,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,234,14,252.49
785,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,150,14,252.49


In [13]:
# X=acc_ds[['year','month','week','day','weekday']]
X = week_d.drop(['acc_nr'], axis=1)
y=week_d['acc_nr']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [29]:
model = RandomForestRegressor(max_depth=6)
model.fit(X_train, y_train)

y_hat = model.predict(X_test)

/Users/franzmornau/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [30]:
model.feature_importances_

array([1.72717762e-02, 4.54229322e-03, 2.32249908e-03, 4.81970601e-03,
       0.00000000e+00, 0.00000000e+00, 1.05381469e-03, 4.28308717e-03,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.33217087e-03,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.18611856e-03,
       0.00000000e+00, 3.69457769e-03, 1.08820417e-03, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 7.19177786e-04, 0.00000000e+00,
       0.00000000e+00, 6.60935901e-04, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 4.81801476e-04, 0.00000000e+00, 5.13197460e-04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 7.21631615e-03,
       0.00000000e+00, 0.00000000e+00, 1.51688148e-02, 4.73190026e-02,
       1.05358639e-02, 0.00000000e+00, 1.97289532e-03, 1.83898581e-03,
       0.00000000e+00, 2.49807021e-02, 0.00000000e+00, 5.57959228e-02,
       2.18191050e-01, 5.70011087e-01, 0.00000000e+00])

In [31]:
model.score(X_train,y_train)

0.5848515130190152

In [32]:
model.score(X_test,y_test)

0.30678443013274803

In [22]:
if True:
    grid = {
    'n_estimators': [10,20,30,100,200,300],
    'criterion': ['mse'],
    'max_depth': [2,3,4,5],
    'min_samples_split': [2,3,4],
    'min_samples_leaf': [1,2,3],
    'min_weight_fraction_leaf': [0.0,0.1,0.2],
#     'max_features'='auto',
#     'max_leaf_nodes'=None,
#     'min_impurity_decrease'=0.0,
#     'min_impurity_split'=None,
#     'bootstrap'=True,
#     'oob_score'=False,
#     'n_jobs'=None,
#     'random_state'=None,
#     'verbose'=0,
#     'warm_start'=False,
    }
else:  # TEST
    grid = {
    'learning_rate': [1],
    'max_depth': [2], 
    'min_samples_leaf': [2],
#     'max_features': ['sqrt', None],
    'n_estimators': [2],
    'random_state': [0]
}
    

from sklearn.model_selection import GridSearchCV
# confusion_score = make_scorer(confusion_rmse, greater_is_better=False)
gbc_grid_cv = GridSearchCV(
    RandomForestRegressor(), 
    grid,
    cv=4,  # number of folds
    return_train_score=True,
    verbose=1, 
    n_jobs=-1)
gbc_grid_cv.fit(X_train, y_train)
# Collapse

Fitting 4 folds for each of 648 candidates, totalling 2592 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 249 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 749 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done 1449 tasks      | elapsed:   19.6s
[Parallel(n_jobs=-1)]: Done 2349 tasks      | elapsed:   32.4s
[Parallel(n_jobs=-1)]: Done 2569 out of 2592 | elapsed:   35.7s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 2592 out of 2592 | elapsed:   36.2s finished


GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [10, 20, 30, 100, 200, 300], 'criterion': ['mse'], 'max_depth': [2, 3, 4, 5], 'min_samples_split': [2, 3, 4], 'min_samples_leaf': [1, 2, 3], 'min_weight_fraction_leaf': [0.0, 0.1, 0.2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [23]:
gbc_grid_cv.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=4,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [24]:
gbc_grid_cv.best_score_

0.40111590505069256

In [23]:
pd.DataFrame(gbc_grid_cv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_max_features,param_min_samples_leaf,param_n_estimators,param_random_state,...,split3_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,mean_train_score,std_train_score
0,0.014000,0.000906,0.001717,0.000358,0.01,3,sqrt,1,30,0,...,0.016264,0.052561,0.023702,1018,0.085244,0.083026,0.086105,0.093652,0.087007,0.003998
1,0.026899,0.002031,0.002592,0.000300,0.01,3,sqrt,1,60,0,...,0.094772,0.134716,0.026554,778,0.174924,0.174276,0.180102,0.189496,0.179699,0.006090
2,0.046446,0.003412,0.002695,0.000810,0.01,3,sqrt,1,90,0,...,0.143797,0.189178,0.029465,591,0.243129,0.241060,0.247264,0.257015,0.247117,0.006135
3,0.054722,0.002069,0.002393,0.000037,0.01,3,sqrt,1,120,0,...,0.184272,0.230570,0.030533,490,0.293013,0.290867,0.297392,0.310273,0.297886,0.007528
4,0.067778,0.003001,0.002443,0.000497,0.01,3,sqrt,1,150,0,...,0.219134,0.267124,0.032381,395,0.338548,0.339922,0.342902,0.357146,0.344629,0.007396
5,0.074605,0.002333,0.002486,0.000363,0.01,3,sqrt,1,180,0,...,0.249669,0.300066,0.034068,325,0.381067,0.382061,0.384874,0.398839,0.386710,0.007140
6,0.080045,0.002980,0.002132,0.000037,0.01,3,sqrt,1,210,0,...,0.273468,0.325513,0.035342,267,0.414422,0.415200,0.418873,0.431930,0.420106,0.007031
7,0.015021,0.000789,0.001711,0.000040,0.01,3,sqrt,2,30,0,...,0.016545,0.052272,0.024318,1019,0.082113,0.081180,0.083414,0.091050,0.084439,0.003898
8,0.022826,0.000469,0.001798,0.000030,0.01,3,sqrt,2,60,0,...,0.094805,0.133331,0.027482,782,0.169236,0.171395,0.175657,0.185264,0.175388,0.006152
9,0.033685,0.000726,0.001811,0.000035,0.01,3,sqrt,2,90,0,...,0.142715,0.187774,0.031160,598,0.234970,0.235727,0.242009,0.250894,0.240900,0.006385
